In [1]:
import pandas as pd

from utils import Coordinates, CleanedColumnNames
from lightcurve import LightCurve, BinnedLightCurve, Transient
from clean import LightCurveCleaner

In [2]:
coords = Coordinates("186.44114","12.66357")
filt = "o"

# test cleaning

In [3]:
transient = Transient(filt=filt, verbose=True)

for i in range(0, 9):
    lc = LightCurve(i, coords, verbose=True)
    if i == 0:
        filename = f"/Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/2023fyq.{filt}.lc.txt"
    else:
        filename = f"/Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/controls/2023fyq_i{i:03d}.{filt}.lc.txt"
    print(i, filename)
    df = pd.read_table(filename, sep="\s+")
    lc.set(df)

    # transient.add(lc, deep=False)
    transient.lcs[lc.control_index] =lc

transient.preprocess()

0 /Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/2023fyq.o.lc.txt
1 /Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/controls/2023fyq_i001.o.lc.txt
2 /Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/controls/2023fyq_i002.o.lc.txt
3 /Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/controls/2023fyq_i003.o.lc.txt
4 /Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/controls/2023fyq_i004.o.lc.txt
5 /Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/controls/2023fyq_i005.o.lc.txt
6 /Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/controls/2023fyq_i006.o.lc.txt
7 /Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/controls/2023fyq_i007.o.lc.txt
8 /Users/sofiarest/Desktop/Supernovae/data_refactor/atclean_input/2023fyq/controls/2023fyq_i008.o.lc.txt
Deleting 0 rows with "duJy"==0 or "uJy"==NaN
Converting flux to magni

In [4]:
print(transient.num_controls)
print(transient.lc_indices)
print(transient.control_lc_indices)
print(transient.colnames)
print()
print(transient.get(2).t.head())
print(transient.get(2).colnames)

8
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3, 4, 5, 6, 7, 8]
-- Required Columns --
mjd: MJD
ra: RA
dec: Dec
mag: m
dmag: dm
flux: uJy
dflux: duJy
filter: F
limiting_mag: mag5sig
-- Optional Columns --
chisquare: chi/N
snr: SNR

            MJD          m  dm   uJy  duJy  F  err  chi/N         RA      Dec  \
0  57205.272537  18.504094 NaN -66.0  48.0  o  0.0   1.09  186.44114  12.6683   
1  57205.273313  18.356290 NaN -94.0  55.0  o  0.0   1.43  186.44114  12.6683   
2  57205.281246  17.990970 NaN  63.0  56.0  o  0.0   0.92  186.44114  12.6683   
3  57205.281906  18.417189 NaN -10.0  52.0  o  0.0   1.59  186.44114  12.6683   
4  57206.277034  18.048890 NaN  69.0  50.0  o  0.0   1.17  186.44114  12.6683   

   ...        y   maj   min   phi  apfit  mag5sig    Sky             Obs Mask  \
0  ...  5145.00  3.98  3.69 -86.4 -0.431    18.28  19.15  02a57205o0017o    0   
1  ...  6115.26  4.03  3.46 -88.4 -0.414    18.32  19.24  02a57205o0018o    0   
2  ...  5144.12  3.95  3.63 -80.8 -0.425    18.

In [5]:
cleaner = LightCurveCleaner(verbose=True)

In [6]:
transient = cleaner.apply_UncertaintyCut(transient)
transient = cleaner.apply_UncertaintyEstimation(transient)
transient, stats_table = cleaner.apply_ChiSquareCut(transient)
previous_flags = 0x1 | 0x2
transient = cleaner.apply_ControlLightCurveCut(transient, previous_flags)

We can increase the typical uncertainties from 23.50 to 30.50 by adding an additional systematic uncertainty of 19.44 in quadrature
New typical uncertainty is 29.77% greater than old typical uncertainty
Applying true uncertainties estimation
Replacing infs with NaNs
Calculating flux/dflux in for 'SNR' column
Replacing infs with NaNs
Calculating flux/dflux in for 'SNR' column
Replacing infs with NaNs
Calculating flux/dflux in for 'SNR' column
Replacing infs with NaNs
Calculating flux/dflux in for 'SNR' column
Replacing infs with NaNs
Calculating flux/dflux in for 'SNR' column
Replacing infs with NaNs
Calculating flux/dflux in for 'SNR' column
Replacing infs with NaNs
Calculating flux/dflux in for 'SNR' column
Replacing infs with NaNs
Calculating flux/dflux in for 'SNR' column
Replacing infs with NaNs
Calculating flux/dflux in for 'SNR' column
Success
The extra noise was added to the uncertainties of the SN light curve and copied to the "duJy_new" column
Calculating loss and contaminatio

In [11]:
print()
print(cleaner.cut_history)

print()
print(transient.get(0).get_percent_flagged(flag=0x2) *100)
print(transient.get(0).t[transient.colnames.dflux_offset].head())
print(transient.get(0).get_percent_flagged(flag=0x1) *100)
print(transient.get(0).get_percent_flagged(flag=0x100) *100)
print(transient.get(0).get_percent_flagged(flag=0x200) *100)
print(transient.get(0).get_percent_flagged(flag=0x400) *100)
print(transient.get(0).get_percent_flagged(flag=0x800) *100)
print(transient.get(0).get_percent_flagged(flag=0x80000) *100)
print(transient.get(0).get_percent_flagged(flag=0x400000) *100)


-- Cut History --
Uncertainty Cut
- 'high_uncertainty' primary flag (0x2): measurement has an uncertainty above 160
True Uncertainties Estimation: We attempt to account for an extra noise source in the data by estimating the true typical uncertainty, deriving the additional systematic uncertainty, and applying this extra noise to the uncertainty column. We also use a temporary, very high PSF chi-square cut value of 20 to eliminate the most egregious outliers from the data before estimating the true uncertainties.
PSF Chi-Square Cut
- 'high_psf_chi_square' primary flag (0x1): measurement has a PSF chi-square above 10
Control Light Curve Cut: For a given SN epoch, we can calculate the 3sigma-clipped average of the corresponding N control flux measurements falling within the same epoch. Given the expectation of control flux consistency with zero, the statistical properties accompanying the 3sigma-clipped average enable us to identify problematic epochs.
- 'bad_epoch' primary flag (0x4000

# test binning

In [8]:
lc = LightCurve(0, coords, verbose=True)
df = pd.read_table("/Users/sofiarest/Desktop/Supernovae/data_refactor/output/misc/2023fyq/2023fyq.o.clean.lc.txt", sep="\s+")
lc.set(df)
lc.t[lc.colnames.mask] = lc.t[lc.colnames.mask].apply(
    lambda x: int(x, 16) if isinstance(x, str) and x.startswith("0x") else int(x)
)

In [9]:
print(lc.colnames.__str__())
lc.t.head()

-- Required Columns --
mjd: MJD
ra: RA
dec: Dec
mag: m
dmag: dm
flux: uJy
dflux: duJy
filter: F
limiting_mag: mag5sig
-- Optional Columns --
chisquare: chi/N
snr: SNR


,MJD,m,dm,uJy,duJy,F,err,chi/N,RA,Dec,...,y,maj,min,phi,apfit,mag5sig,Sky,Obs,Mask,duJy_new
0,57205.272537,18.550302,NaN,-28,46,o,0,1.92,186.44114,12.66357,...,5135.81,3.98,3.69,-86.4,-0.431,18.28,19.15,02a57205o0017o,0,49.940796
1,57205.273313,18.396507,NaN,-93,53,o,0,2.50,186.44114,12.66357,...,6106.09,4.03,3.46,-88.4,-0.414,18.32,19.24,02a57205o0018o,524288,56.454257
2,57205.281246,18.417189,NaN,-117,52,o,0,1.46,186.44114,12.66357,...,5134.92,3.95,3.63,-80.8,-0.424,18.27,19.10,02a57205o0029o,524288,55.516512
3,57205.281906,18.024380,NaN,38,62,o,0,1.76,186.44114,12.66357,...,6105.49,3.97,3.44,-89.4,-0.429,18.30,19.20,02a57205o0030o,0,64.977558
4,57206.277034,18.438271,NaN,-88,51,o,0,1.53,186.44114,12.66357,...,5130.67,4.73,3.71,-47.1,-0.423,18.29,19.39,02a57206o0007o,524288,54.580977


In [10]:
previous_flags = 0x1|0x2|0x400000
binned_lc = lc.get_BinnedLightCurve(previous_flags)

Now averaging SN light curve


KeyboardInterrupt: 

In [ ]:
df = binned_lc.t.copy()
df["Mask"] = df["Mask"].apply(lambda x: hex(x) if x != 0 else f"0x0000")
print(df.head())
df.to_string("test.txt", index=False)